In [7]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user

In [8]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='World_Sep-Nov-23.csv'
isPy=False 
isPriceExcel=False

# Specify Period and Timefame to caculate fund performance and

In [9]:
from_str='2023-08-01'
to_str='2023-11-30' 
timeframe='M'
#timeframe='W-FRI'

# Input data

In [10]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [11]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\World_Sep-Nov-23.csv
D:\PythonDev\MyQuantFinProject\Data\World_Sep-Nov-23.csv


# Read csv file and extract and transform data as need

In [12]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df.loc[from_str:to_str,['symbol','price']]
print(df.info())

list_symbol=df['symbol'].unique().tolist() 
print(list_symbol)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336 entries, 2023-08-01 to 2023-11-28
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  336 non-null    object 
 1   price   336 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.9+ KB
None
['AAXJ', 'QQQ', 'ACWI', 'SPY']


In [13]:
df.head(len(list_symbol)*3)

,symbol,price
date,,
2023-08-01,AAXJ,69.70
2023-08-01,QQQ,382.79
2023-08-01,ACWI,98.70
2023-08-01,SPY,456.48
2023-08-02,AAXJ,67.90
2023-08-02,QQQ,374.39
2023-08-02,SPY,450.13
2023-08-02,ACWI,97.07
2023-08-03,AAXJ,68.43


In [14]:
df.tail(len(list_symbol)*3)

,symbol,price
date,,
2023-11-24,QQQ,389.51
2023-11-24,SPY,455.30
2023-11-24,AAXJ,65.74
2023-11-24,ACWI,97.88
2023-11-27,QQQ,389.17
2023-11-27,SPY,454.48
2023-11-27,ACWI,97.62
2023-11-27,AAXJ,65.38
2023-11-28,ACWI,97.78


# Basic

# Resample data and calculate Percentage of change

In [15]:
colsROC=['symbol','date','pct_change']
df_Pct_Price=pd.DataFrame(columns=colsROC)
df_Pct_Price

,symbol,date,pct_change


In [16]:
for symbol_name in list_symbol:

    dfx_symbol=df.query('symbol==@symbol_name')
    print(f"===================================={symbol_name}====================================")

    dfx_px=dfx_symbol.resample(timeframe).last()
    dfx_px=dfx_px.reset_index()
    print(f"List {timeframe} Price")
    print(dfx_px)

    dfx_px['pct_change']=dfx_px['price'].pct_change()*100
    dfx_px['pct_change']=dfx_px['pct_change'].round(2)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    dfx_px=dfx_px[colsROC]
    
    print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

====================================AAXJ====================================
List M Price
        date symbol  price
0 2023-08-31   AAXJ  65.58
1 2023-09-30   AAXJ  63.50
2 2023-10-31   AAXJ  61.41
3 2023-11-30   AAXJ  65.84
Calculate ROC of AAXJ
  symbol       date  pct_change
1   AAXJ 2023-09-30       -3.17
2   AAXJ 2023-10-31       -3.29
3   AAXJ 2023-11-30        7.21
====================================QQQ====================================
List M Price
        date symbol   price
0 2023-08-31    QQQ  377.99
1 2023-09-30    QQQ  358.27
2 2023-10-31    QQQ  350.87
3 2023-11-30    QQQ  390.19
Calculate ROC of QQQ
  symbol       date  pct_change
1    QQQ 2023-09-30       -5.22
2    QQQ 2023-10-31       -2.07
3    QQQ 2023-11-30       11.21
====================================ACWI====================================
List M Price
        date symbol  price
0 2023-08-31   ACWI  96.50
1 2023-09-30   ACWI  92.37
2 2023-10-31   ACWI  90.02
3 2023-11-30   ACWI  97.78
Calculate ROC of ACWI


In [17]:
print("==============================================================")
print("List calculate Percentage of change")
print(df_Pct_Price)

List calculate Percentage of change
   symbol       date  pct_change
0    AAXJ 2023-09-30       -3.17
1    AAXJ 2023-10-31       -3.29
2    AAXJ 2023-11-30        7.21
3     QQQ 2023-09-30       -5.22
4     QQQ 2023-10-31       -2.07
5     QQQ 2023-11-30       11.21
6    ACWI 2023-09-30       -4.28
7    ACWI 2023-10-31       -2.54
8    ACWI 2023-11-30        8.62
9     SPY 2023-09-30       -5.08
10    SPY 2023-10-31       -2.17
11    SPY 2023-11-30        8.78


# Sum percentage of fund return

In [18]:
print("==============================================================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[['pct_change']].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price=dfSum_Pct_Price.sort_values(by="pct_change",ascending=False)
dfSum_Pct_Price["pct_change"]=dfSum_Pct_Price["pct_change"].round(1)
print(dfSum_Pct_Price)

Sum percentage of fund return
  symbol  pct_change
2    QQQ         3.9
1   ACWI         1.8
3    SPY         1.5
0   AAXJ         0.8


In [19]:
# writer=pd.ExcelWriter( "BasicFundPerformanceReport.xlsx",engine='openpyxl') 

# df_Pct_Price.to_csv(f"ETF-{timeframe}-Return.csv",index=False)
# dfSum_Pct_Price.to_csv(f"ETF-{timeframe}-TotalReturn.csv",index=False)


# Advance

# Pivot Percent Return To Dataframe

In [20]:
print("==============================================================")
print("Pivot Percent Return To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values='pct_change')
# print(dfPct.info())
print(dfPct)

Pivot Percent Return To Dataframe
symbol      AAXJ  ACWI    QQQ   SPY
date                               
2023-09-30 -3.17 -4.28  -5.22 -5.08
2023-10-31 -3.29 -2.54  -2.07 -2.17
2023-11-30  7.21  8.62  11.21  8.78


# Take Percent Return to Find Rank

In [21]:
print("==============================================================")
print("Take Percent Return to Find Rank")
dfRank=dfPct[dfPct.columns.tolist()].rank(method='max',axis=1)
print(dfRank)

Take Percent Return to Find Rank
symbol      AAXJ  ACWI  QQQ  SPY
date                            
2023-09-30   4.0   3.0  1.0  2.0
2023-10-31   1.0   2.0  4.0  3.0
2023-11-30   1.0   2.0  4.0  3.0


# Combine Both return and Rank

In [22]:
print("==============================================================")
print("Combine Both return and Rank")
dfSummary=pd.concat([\
                     dfPct.sum().to_frame(name='sum_return'),\
                     dfRank.sum().to_frame(name='sum_rank'),\
                    ],axis=1)
dfSummary.fillna(value=0,inplace=True)
print(dfSummary)

Combine Both return and Rank
        sum_return  sum_rank
symbol                      
AAXJ          0.75       6.0
ACWI          1.80       7.0
QQQ           3.92       9.0
SPY           1.53       8.0


# Summarize Total Score 

In [23]:
print("==============================================================")
print("Summarize Total Score ")
dfFinalScore=dfSummary.rank(method='max',axis=0)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore.sort_values(by='TotalScore',ascending=False,inplace=True)
print(dfFinalScore)

Summarize Total Score 
        sum_return  sum_rank  TotalScore
symbol                                  
QQQ            4.0       4.0         8.0
ACWI           3.0       2.0         5.0
SPY            2.0       3.0         5.0
AAXJ           1.0       1.0         2.0


In [24]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfSum_Pct_Price.to_excel(writer, sheet_name="Return")
        dfRank.to_excel(writer, sheet_name="RANK")
        dfSummary.to_excel(writer, sheet_name="Summary")
        dfFinalScore.to_excel(writer, sheet_name="FinalScore")